# Seminar: Computational Mathematics
$\newcommand{\vect}[1]{\boldsymbol{#1}}$

- Simulation of a fluid flow based on the incompressible Navier-Stokes equations
- Variable geometry, with interactive deformation during simulation
- (Reduced Basis Methods)
- Implementation with NGSolve

In [37]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

from time import sleep

import ipywidgets as widgets
from ipywidgets import interact, fixed

## Geometry and Mesh

In [38]:
square = MoveTo(0,0).Rectangle(5,1).Face()
hole = Circle((2.5,0.5), 0.2).Face()
hole.edges.name = "circle"
square.edges.Max(Y).name = "top"
square.edges.Min(Y).name = "bottom"
square.edges.Min(X).name = "inlet"
square.edges.Max(X).name = "outlet"
shape = square - hole
DrawGeo(shape)

WebGuiWidget(value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3, 'mesh_center': [2.5, 0.5000000000000003, 0…

BaseWebGuiScene

In [39]:
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.3)).Curve(3)
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

BaseWebGuiScene

## Deformation (Top, Bottom, Circle)

In [40]:
fes_def = VectorH1(mesh, order=3, dirichlet=".*")

We create a smooth deformation by solving a Poisson equation in discrete form $ A \ u = f $ with $ f= 0 $.

The bilinear form $A$ includes both the FreeDofs and the fixed Dirichlet DOFs:
$$ A \ u = \begin{pmatrix} A_f & A_D \\ ... & ... \end{pmatrix} \begin{pmatrix} u_f \\ u_D \end{pmatrix} = 0 $$

In [41]:
u_def, v_def = fes_def.TnT()
a_def = BilinearForm(InnerProduct(grad(u_def), grad(v_def))*dx).Assemble()
A_def_inv = a_def.mat.Inverse(freedofs=fes_def.FreeDofs())

To get $u$, we first set a fixed deformation at the boundary, `gf_def`, corresponding to $u_D$.

In [42]:
gf_def = GridFunction(fes_def) # this will hold all deformations
gf_def_top = GridFunction(fes_def) # upper boundary
gf_def_bot = GridFunction(fes_def) # lower boundary
gf_def_rad = GridFunction(fes_def) # radius of hole

gf_def_top.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("top"))
gf_def_bot.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("bottom"))
gf_def_rad.Set(5*CF((x-2.5, y-0.5)), definedon=mesh.Boundaries("circle"))

From this we obtain $f_D$, using the `*`-operator (matrix-vector product):
$ f_D = -A \begin{pmatrix} 0 \\ u_D \end{pmatrix} $ \
Finally, we solve for $u_f$ by using only the `FreeDofs` $$ u_f = A_f^{-1} \cdot f_D $$ with the `@`-operator, and add this to the total deformation.

In [43]:
gf_def_top.vec.data -= A_def_inv@a_def.mat*gf_def_top.vec    # Matrix @ Matrix * Vector
gf_def_bot.vec.data -= A_def_inv@a_def.mat*gf_def_bot.vec
gf_def_rad.vec.data -= A_def_inv@a_def.mat*gf_def_rad.vec

In [44]:
# initial values for deformations
delta_top = Parameter(-0.005)
delta_bot = Parameter(0.005)
delta_rad = Parameter(0.02)

# combine all deformations into one grid function
gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                  delta_bot.Get() * gf_def_bot.vec + \
                  delta_rad.Get() * gf_def_rad.vec

In [45]:
Draw(gf_def, deformation=gf_def)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

## Deformation (Bending)

With the previous method we had to specify the full displacement vector $u$ on the Dirichlet boundary. Now we want to set only one component ($u_y$) on one of the sides (outlet) and let the displacement on the rest of the boundary be calculated automatically by, again, solving a Poisson problem.

To achieve this, we can use a mixed formulation, applying Dirichlet boundary conditions only on one component, while the remaining components will have zero Neumann BC. Test functions $v$ are not required to be zero on the new Dirichlet boundary, which adds a boundary term to the equation:
$$
\int_\Omega \nabla u \nabla v - \int_{\Gamma_D} \frac{\partial u_y}{\partial n} v_y = \int_\Omega f v
$$

$$ \begin{array}{ccccll} u_y = u_D & \text{on} \ \Gamma_{D} \end{array} $$
We introduce a new variable $$\lambda = -\frac{\partial u_y}{\partial n}$$
and the mixed problem ($f=0$):

Find $u \in V := [H^1(\Omega)]^d$ and $\lambda \in Q$ such that
$$
\begin{array}{ccccll}
\int_\Omega \nabla u \nabla v & + & \int_{\Gamma_D} \lambda \ v_y & = & 0 & \forall \, v \in V \\
\int_{\Gamma_D} \mu \ u_y & & & = & \int_{\Gamma_D} \mu \ u_D & \forall \, \mu \in Q
\end{array}
$$

For $u_D=1$ it is enough to choose a 1-dimensional space: $Q = \mathbb{R}$.
In general one would use $Q := H^{-1/2}(\Gamma_D)$ or $L^2$.

However, since the Poisson problem is still decoupled for the different coordinates, there is still no displacement in x-direction. To get a more realistic deformation, we symmetrize the gradients:
$$ \text{Sym}(\nabla u) = \frac{1}{2} (\nabla u + \nabla u^T) =: \varepsilon(u) $$
This is like the (linearized) mechanical strain tensor $\varepsilon$, and we get $\varepsilon(u) : \varepsilon(v)$ in the first integral.

In [48]:
mesh.SetDeformation(None)

fes_def_2 = VectorH1(mesh, order=3, dirichlet="inlet")
fes_number = NumberSpace(mesh)
X_def = fes_def_2 * fes_number

(u_def_2, lam), (v_def_2, mu) = X_def.TnT()

In [50]:
a_def_2 = BilinearForm(X_def)
a_def_2 += InnerProduct(Sym(grad(u_def_2)), Sym(grad(v_def_2)))*dx
# Same as:
#a_def_2 += InnerProduct(0.5*(grad(u_def_2) + grad(u_def_2).trans), 0.5*(grad(v_def_2) + grad(v_def_2).trans))*dx
a_def_2 += (u_def_2[1]*mu + v_def_2[1]*lam)*ds("outlet")
a_def_2.Assemble()

A_def_inv_2 = a_def_2.mat.Inverse(freedofs=X_def.FreeDofs())

In [51]:
gf_mean_out = GridFunction(X_def)
l_def_2 = LinearForm(X_def)
l_def_2 += 1*mu*ds("outlet")
l_def_2.Assemble()

gf_mean_out.vec.data = A_def_inv_2*l_def_2.vec

In [52]:
delta_out = Parameter(0.3)

# add deformation to "global" deformation function
gf_def.vec.data += delta_out.Get()*gf_mean_out.components[0].vec

In [53]:
#mesh.SetDeformation(gf_def)

In [54]:
#Draw(gf_mean_out.components[0].components[0])
Draw(gf_def, deformation=gf_def)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene